In [1]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langgraph.graph import END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import os
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from pydantic import BaseModel, Field
from langchain.agents import Tool
from typing import Optional
from langchain_core.tools import StructuredTool
import json
from langchain.output_parsers.structured import (
StructuredOutputParser, ResponseSchema
)


from datetime import datetime

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.structured import (
StructuredOutputParser, ResponseSchema
)

In [2]:
from langchain_aws import ChatBedrock
import os
from dotenv import load_dotenv

load_dotenv()
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_ACCESS_KEY= os.getenv("AWS_ACCESS_KEY")
llm = ChatBedrock(
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    model_kwargs=dict(temperature=0)
)


In [5]:
result = llm.invoke("Why is gpu better than cpu in ai and mahcine learning?")
print(type(result))
print(result)

<class 'langchain_core.messages.ai.AIMessage'>
content="GPUs (Graphics Processing Units) are better suited for AI and machine learning tasks compared to CPUs (Central Processing Units) for several key reasons:\n\n1. Parallel Processing:\n- GPUs have thousands of smaller cores designed for parallel processing\n- AI/ML tasks often involve many simultaneous calculations that can be performed in parallel\n- CPUs have fewer cores (typically 4-16) optimized for sequential processing\n\n2. Matrix Operations:\n- AI/ML involves heavy matrix multiplication and vector operations\n- GPUs are specifically designed to handle these types of calculations efficiently\n- Modern GPUs include specialized tensor cores for deep learning operations\n\n3. Memory Bandwidth:\n- GPUs have higher memory bandwidth\n- Can transfer large amounts of data quickly\n- Important for handling large datasets and model parameters\n\n4. Specialized Architecture:\n- Built for handling repetitive, simple calculations\n- Optimi

In [ ]:

def question_extractor_chain(state):
    print("starting now\n\n\n")

    question_explorer_prompt = ChatPromptTemplate.from_template(
"""
Role: Search Engine Question Optimizer Robot

Scenario:
User has a question that he/she wants to know from the internet not from you.

Task: Base on user question, try to dive deeper to the intent, the technical aspects of the query, and optimize user's question for search engine.
You are not responsible in answering user's question. You need to generate 10 search query based on the user's question
that allows user to obtain the best informtaion from the internet through the search engine, NOT FROM YOU. 
The generated questions should not enquire user to input information, instead, they should be ready for search engine. 
Optimize all the questions for search engine that enables user to obtain high quality information. 

This is the user's question: {question}

Instructions:
Think step by step to generate 10 search engine queries at the best of your ability.
Assume user will copy and paste directly without any changes, so there shouldn't be [] variable that allows user to change.

Output Format: No greeting, no bold text, no Italic text, just plain text in string, no number index
"""
    ).partial(time=datetime.now())


    question_explorer_chain = question_explorer_prompt | llm
    if "list_of_questions" in state:
        temp_list = state["list_of_questions"]
    else:
        temp_list = []

    for question in state["org_questions"]:
        result = question_explorer_chain.invoke(question)
        temp_list.extend(result["list_of_queries"])

    return {"list_of_questions": temp_list}